# exe 3.1 p.91
Your task is to correctly assign the weights from an instance of SelfAttention_v2 to
an instance of SelfAttention_v1. To do this, you need to understand the
relationship between the weights in both versions. (Hint: nn.Linear stores the
weight matrix in a transposed form.) After the assignment, you should observe that
both instances produce the same outputs.

In [2]:
import torch.nn as nn
import torch

In [5]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [86]:
torch.manual_seed(1234)

In [87]:
"""
inputs: A(6x3)
weight matrices: WQ(3,2), WK(3,2), WV(3,2) # random init, 3 input embedding size, 2 chosen emdedding dimensions
K,V: K(6,2) = A * WK, V = A * WV
QY[i]: QY[i](1,2) = A[i](1,3) * Q(3,2) # QY is query for element i, i.e attention given to token i by all other tokens
attention scores: S(1,6) = QY[i](1,2) * K.T(2,6)
embedding dim: ED = K.shape[1] # 
attention weights [i]: AW(1,6) = softmax(S/sqrt(ED), dim=-1). aka. softmax on rows. we divide by square root of ED (why ??)
context vector [i]: AW(1,6) * V(6,2) 
"""

'\ninputs: A(6x3)\nweight matrices: WQ(3,2), WK(3,2), WV(3,2) # random init, 3 input embedding size, 2 chosen emdedding dimensions\nK,V: K(6,2) = A * WK, V = A * WV\nQY[i]: QY[i](1,2) = A[i](1,3) * Q(3,2) # QY is query for element i, i.e attention given to token i by all other tokens\nattention scores: S(1,6) = QY[i](1,2) * K.T(2,6)\nembedding dim: ED = K.shape[1] # \nattention weights [i]: AW(1,6) = softmax(S/sqrt(ED), dim=-1). aka. softmax on rows. we divide by square root of ED (why ??)\ncontext vector [i]: AW(1,6) * V(6,2) \n'

In [88]:
class SelfAttentioV1(nn.Module):
    def __init__(self, d_in, d_out) -> None:
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_keys = nn.Parameter(torch.rand(d_in, d_out))
        self.W_values = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x: torch.Tensor):
        queries = x @ self.W_query
        keys = x @ self.W_keys
        values = x @ self.W_values

        attn_score = queries @ keys.T
        attn_weights = torch.softmax(attn_score / keys.shape[-1]**0.5, dim=-1)
        contextvec = attn_weights @ values
        return contextvec

In [89]:
class SelfAttentioV2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False) -> None:
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, qkv_bias)
        self.W_keys = nn.Linear(d_in, d_out, qkv_bias)
        self.W_values = nn.Linear(d_in, d_out, qkv_bias)

    def forward(self, x: torch.Tensor):
        queries = self.W_query(x)
        keys = self.W_keys(x)
        values = self.W_values(x)

        attn_score = queries @ keys.T
        attn_weights = torch.softmax(attn_score / keys.shape[-1]**0.5, dim=-1)
        contextvec = attn_weights @ values
        return contextvec

In [90]:
d_in = inputs.shape[1]
d_out = 2
sav1 = SelfAttentioV1(d_in=d_in, d_out=d_out)
sav2 = SelfAttentioV2(d_in=d_in, d_out=d_out)

In [91]:
sav1.forward(inputs)

tensor([[0.6641, 1.1324],
        [0.6694, 1.1453],
        [0.6691, 1.1446],
        [0.6592, 1.1247],
        [0.6570, 1.1190],
        [0.6632, 1.1333]], grad_fn=<MmBackward0>)

In [92]:
sav2(inputs) # different output to sav1 because different initial weights

tensor([[0.1478, 0.0307],
        [0.1502, 0.0307],
        [0.1502, 0.0308],
        [0.1492, 0.0305],
        [0.1498, 0.0320],
        [0.1491, 0.0299]], grad_fn=<MmBackward0>)

In [93]:
sav2.W_keys.weight

Parameter containing:
tensor([[ 0.5387,  0.2771, -0.0558],
        [-0.0280,  0.3282, -0.4013]], requires_grad=True)

In [94]:
with torch.no_grad():
    sav2.W_query.weight.copy_(sav1.W_query.T) # transposing because linear inverts d_in, d_out
    sav2.W_keys.weight.copy_(sav1.W_keys.T)
    sav2.W_values.weight.copy_(sav1.W_values.T)

In [106]:
sav1.forward(inputs).sum()

tensor(10.7813, grad_fn=<SumBackward0>)

In [107]:
sav2.forward(inputs).sum()

tensor(10.7813, grad_fn=<SumBackward0>)

In [108]:
sav2.forward(inputs).sum().eq(sav1.forward(inputs).sum())

tensor(True)